# A3: A\*, IDS, and Effective Branching Factor

## Introduction

For this assignment, we implemented the Recursive Best-First Search implementation of the A\* algorithm given in class.  We named this function `aStarSearch`.  Also included in this notebook is the `iterativeDeepeningSearch` functions from Assignment 2. A new function named `ebf` was developed that returns an estimate of the effective branching factor for a search algorithm applied to a search problem.

So, the functions developed in this assignment are as follows:

   - `aStarSearch(startState, actionsF, takeActionF, goalTestF, hF)`
   - `iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth)`
   - `ebf(nNodes, depth, precision=0.01)`, returns number of nodes expanded and depth reached during a search.

We also applied `iterativeDeepeningSearch` and `aStarSearch` to several eight-tile sliding puzzle
problems. For this, we included the following functions, from Assignment 2:

  * `actionsF_8p(state)`: returns a list of up to four valid actions that can be applied in `state`. With each action include a step cost of 1. For example, if all four actions are possible from this state, return [('left', 1), ('right', 1), ('up', 1), ('down', 1)].
  * `takeActionF_8p(state, action)`: return the state that results from applying `action` in `state` and the cost of the one step,
  
plus the following function for the eight-tile puzzle:

  * `goalTestF_8p(state, goal)`
  
In the end of this assignemnt, we will compare their results by displayng solution path, depth, number of nodes 
generated, and the effective branching factor, and discuss the results.  We achieve this by defining the following function that prints the table as shown in the example below.

   - runExperiment(goalState1, goalState2, goalState3, [h1, h2, h3])
  

Apply all four algorithms (`iterativeDeepeningSearch` plus `aStarSearch` with the three heuristic
functions) to three eight-tile puzzle problems with start state

$$
\begin{array}{ccc}
1 & 2 & 3\\
4 & 0 & 5\\
6 & 7 & 8
\end{array}
$$

and these three goal states.

$$
\begin{array}{ccccccccccc}
1 & 2 & 3  & ~~~~ & 1 & 2 & 3  &  ~~~~ & 1 & 0 &  3\\
4 & 0 & 5  & & 4 & 5 & 8  & & 4 & 5 & 8\\
6 & 7 & 8 &  & 6 & 0 & 7  & & 2 & 6 & 7
\end{array}
$$

## Heuristic Functions

For `aStarSearch`function the following three heuristic functions were designed.

  * `h1_8p(state, goal)`: $h(state, goal) = 0$, for all states $state$ and all goal states $goal$,
  * `h2_8p(state, goal)`: $h(state, goal) = m$, where $m$ is the Manhattan distance that the blank is from its goal position,
  * `h3_8p(state, goal)`: $h(state, goal) = ?$, that you define.  It must be admissible, and not constant for all states.


           [1, 2, 3, 4, 0, 5, 6, 7, 8]    [1, 2, 3, 4, 5, 8, 6, 0, 7]    [1, 0, 3, 4, 5, 8, 2, 6, 7] 
    Algorithm    Depth  Nodes  EBF              Depth  Nodes  EBF              Depth  Nodes  EBF          
         IDS       0      0  0.000                3     43  3.086               11 225850  2.954         
        A*h1       0      0  0.000                3    116  4.488               11 643246  3.263         
        A*h2       0      0  0.000                3     51  3.297               11 100046  2.733         



In [1]:
# from A3mysolution import *

First, some example output for the ebf function.

In [2]:
import numpy as np
import math
import copy
import pandas as pd

## Effective Branch Factor
The formulae for Effective Branching Factor (EBF) is given by:
                $$ \frac{1-b^{d+1}}{1-b} = Nodes$$ 
                where b is the Effective Branch Factor 
                       d is the depth of search
                       
This equation can be solved using 2 methods:
* Binary Search
* Newton-Raphson Method

For the ebf function defined below, the ebf equation has been solved using the Newton-Raphson method of obtaining local minima (zero) of an equation. Kindly refer to (https://en.wikipedia.org/wiki/Newton%27s_method) for more information about the Newton-raphson equation

In [3]:
nodes=0
netdepth=0

def ebf(nNodes, depth, precision=0.01):
    if depth==0:
        if nNodes==1:
            return 1.0
        else:
            return 0.0
    global nodes
    nodes=nNodes
    global netdepth
    netdepth=depth
    precision=precision
    x0=nNodes**(1/(depth+1))
    result = solve(quadratic, x0, precision) 
    return result

def derivative(f, x, h):
      return (f(x+h) - f(x-h)) / (2.0*h)  

def quadratic(x):
    return ((1-x**(netdepth +1))/(1-x))-nodes     

def solve(f, x0, h):
    lastX = x0
    nextX = lastX + 10* h  
    while (abs(lastX - nextX) > h):  
        newY = f(nextX)                        
        lastX = nextX
        nextX = lastX - newY / derivative(f, lastX, h)  
    return nextX        

### EBF Examples

In [4]:
ebf(10, 3)

1.6608024572821003

In [5]:
ebf(1, 0)

1.0

In [6]:
ebf(0,0)

0.0

In [7]:
ebf(2, 1)

1.0000000000000007

In [8]:
ebf(2, 1, precision=0.000001)

0.9999999999659235

In [9]:
ebf(200000, 5)

11.275597027190864

In [10]:
ebf(200000, 50)

1.2374964630891387

### Function Description
The following functions have been imported from the Solutions for Assignment 2 posted by Prof. Charles Anderson, CSU CS 440 (http://www.cs.colostate.edu/~anderson/cs440/doku.php?id=schedule)
  * `actionsF_8p(state)`: returns a list of up to four valid actions that can be applied in `state`. With each action include a step cost of 1. For example, if all four actions are possible from this state, return [('left', 1), ('right', 1), ('up', 1), ('down', 1)].
  * `takeActionF_8p(state, action)`: return the state that results from applying `action` in `state` and the cost of the one step,

In [11]:
def findBlank_8p(s):
    return iTorc(s.index(0))

def actionsF_8p(state):
    r, c = findBlank_8p(state)
    actions = []
    if c > 0:
        actions.append(('left',1))
    if c < 2:
        actions.append(('right',1))
    if r > 0:
        actions.append(('up',1))
    if r < 2:
        actions.append(('down',1))
    return actions


def takeActionF_8p(state, action):
    import copy
    state = copy.deepcopy(state)
    r, c = findBlank_8p(state)
    dr = dc = 0
    if action == (('left',1)):
        dc -= 1
    elif action == (('right',1)):
        dc += 1
    elif action == (('up',1)):
        dr -= 1
    elif action == (('down',1)):
        dr += 1
    newr, newc = r+dr, c+dc
    setTile(state, r, c, getTile(state, newr, newc))
    setTile(state, newr, newc, 0)
    return (state,1)

def setTile(state, row, col, tile):
    state[rcToi(row, col)] = tile
    return state


def getTile(state, row, col):
    return state[rcToi(row, col)]

def rcToi(row, col):
    return row*3+col


def iTorc(i):
    row = i // 3
    col = i - row*3
    return (row, col)

### Function Description
This function performs `Iterative Deeeping` Search and has been included from Assigment 2, with some modifications. 

In [12]:
COUNT = 0

def incCount():
    global COUNT
    COUNT += 1
    
def getNodeCount():
    global COUNT
    tmp = COUNT
    COUNT = 0
    return tmp

def iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth):
    global COUNT
    COUNT=0
    for depth in range(maxDepth):
        result = depthLimitedSearch(startState, goalState, actionsF, takeActionF, depth)
        if result == 'failure':
            return 'failure'
        if result != 'cutoff':
            result=[startState]+result 
            return result
    return 'cutoff'

def depthLimitedSearch(state, goalState, actionsF, takeActionF, depthLimit):
    if state == goalState:
        return []
    if depthLimit == 0:
        return 'cutoff'
    cutoffOccurred = False
    for action in actionsF(state):
        global COUNT
        COUNT+=1
        childState,_ = takeActionF(state, action)
        result = depthLimitedSearch(childState, goalState, actionsF, takeActionF, depthLimit-1)
        if result == 'cutoff':
            cutoffOccurred = True      
        elif result != 'failure':
            result=[childState]+result
            return result
    if cutoffOccurred == True:
        return 'cutoff'
    else:
        return 'failure'

In [13]:
def actionsF_simple(state):
    succs = {'a': ['b', 'c'], 'b':['e', 'f', 'g'], 'b':['a'], 'c':['h'], 'h':['i'], 'i':['j', 'k', 'l'], 'k':['z']}
    return [(s, 1) for s in succs.get(state, [])]

def takeActionF_simple(state, action):
    return action

def goalTestF_simple(state, goal):
    return state == goal

def h_simple(state, goal):
    return 1

In [14]:
def goalTestF_8p(state, goal):
    return state==goal

### Function Description
For `aStarSearch`function the following three heuristic functions were designed.

  * `h1_8p(state, goal)`: $h(state, goal) = 0$, for all states $state$ and all goal states $goal$,
  * `h2_8p(state, goal)`: $h(state, goal) = m$, where $m$ is the Manhattan distance that the blank is from its goal position,
  * `h3_8p(state, goal)`: $h(state, goal) = ?$, that you define.  It must be admissible, and not constant for all states.
  
The third heuristic function (h3_8p) is a custom implementation that is admissible. This function determines how many tiles in the startState are different than those in the Goal State. This means if the start is the goal the difference is zero.

In [15]:
def h1_8p(state, goal):
    return 0

In [16]:
def h2_8p(state, goal):
    x=findBlank_8p(state)
    y=findBlank_8p(goal)
    return abs(y[0]-x[0]) + abs(y[1]-x[1])

In [17]:
def h3_8p(state, goal):
    x = 0
    for i in range(len(state)):
        if state[i] != goal[i]:
            x = x + 1          
    return x

In [18]:
actions = actionsF_simple('a')
actions

[('b', 1), ('c', 1)]

In [19]:
takeActionF_simple('a', actions[0])

('b', 1)

In [20]:
goalTestF_simple('a', 'a')

True

In [21]:
h_simple('a', 'z')

1

In [22]:
iterativeDeepeningSearch('a', 'z', actionsF_simple, takeActionF_simple, 10)

['a', 'c', 'h', 'i', 'k', 'z']

### Function Description
This function performs the Recursive Best-First Seatch and has been borrowed from the the class notes of Prof. Charles Anderson, for CSU CS 440 (http://nbviewer.jupyter.org/url/www.cs.colostate.edu/~anderson/cs440/notebooks/07%20Informed%20Search.ipynb). It has been slightly modified to extract node and depth values for any given A Star Search

In [23]:
NCOUNT = 0

def getNCount():
    global NCOUNT
    tmp = NCOUNT
    NCOUNT = 0
    return tmp

class Node:
    def __init__(self, state, f=0, g=0 ,h=0):
        self.state = state
        self.f = f
        self.g = g
        self.h = h
    def __repr__(self):
        return "Node(" + repr(self.state) + ", f=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"

def aStarSearch(startState, actionsF, takeActionF, goalTestF, hF):
    h = hF(startState)
    startNode = Node(state=startState, f=0+h, g=0, h=h)
    return aStarSearchHelper(startNode, actionsF, takeActionF, goalTestF, hF, float('inf'))

def aStarSearchHelper(parentNode, actionsF, takeActionF, goalTestF, hF, fmax):
    global NCOUNT
    if goalTestF(parentNode.state):
        return ([parentNode.state], parentNode.g)
    ## Construct list of children nodes with f, g, and h values
    actions = actionsF(parentNode.state)
    if not actions:
        return ("failure", float('inf'))
    children = []
    for action in actions:
        NCOUNT += 1
        (childState,stepCost) = takeActionF(parentNode.state, action)
        h = hF(childState)
        g = parentNode.g + stepCost
        f = max(h+g, parentNode.f)
        childNode = Node(state=childState, f=f, g=g, h=h)
        children.append(childNode)
    while True:
        # find best child
        children.sort(key = lambda n: n.f) # sort by f value
        bestChild = children[0]
        if bestChild.f > fmax:
            return ("failure",bestChild.f)
        # next lowest f value
        alternativef = children[1].f if len(children) > 1 else float('inf')
        # expand best child, reassign its f value to be returned value
        result,bestChild.f = aStarSearchHelper(bestChild, actionsF, takeActionF, goalTestF,
                                        hF, min(fmax,alternativef))
        if result is not "failure":               
            result.insert(0,parentNode.state)     
            return (result, bestChild.f)     

In [24]:
aStarSearch('a',actionsF_simple, takeActionF_simple,
            lambda s: goalTestF_simple(s, 'z'),
            lambda s: h_simple(s, 'z'))

(['a', 'c', 'h', 'i', 'k', 'z'], 5)

In [25]:
startState = [1,2,3, 4,0,5, 6,7,8]

goalState =  [1,2,3, 4,0,5, 6,7,8]
goalState2 = [1,2,3, 4,5,8, 6,0,7]
goalState3 = [1,0,3, 4,5,8, 2,6,7]

### Function Description
This section implements the following function:
* runExperiment(goalState1, goalState2, goalState3,h=[h1_8p, h2_8p, h3_8p]):
For a given set of parameters:
* The function collects IDS Data
* The function collects  A Star Search data
* Prints the given data

In [26]:
def getIDSData(goal):
    res = iterativeDeepeningSearch(startState, goal,actionsF_8p,takeActionF_8p, 20)
    depth = len(res) - 1 
    nodes = getNodeCount()
    branchf = ebf(nodes, depth)
    
    return depth, nodes, branchf

def getAStarData(goal, hfunc):
    
    res = aStarSearch(startState,actionsF_8p, takeActionF_8p,
            lambda s: goalTestF_8p(s, goal),lambda s: hfunc(s, goal))
    depth = len(res[0]) - 1 
    nodes = getNCount()
    branchf = ebf(nodes, depth)
    
    return depth, nodes, branchf

def runExperiment(goalState1, goalState2, goalState3,h=[h1_8p, h2_8p, h3_8p]):
    mformat = '''
            {}    {}    {} 
Algorithm    Depth  Nodes       EBF              Depth  Nodes       EBF              Depth  Nodes       EBF          
     IDS       {}      {}       {:.3f}              {}      {}      {:.3f}              {}      {}      {:.3f}         
    A*h1       {}      {}       {:.3f}              {}      {}      {:.3f}              {}      {}      {:.3f}
    A*h2       {}      {}       {:.3f}              {}      {}      {:.3f}              {}      {}      {:.3f}
    A*h3       {}      {}       {:.3f}              {}      {}      {:.3f}              {}      {}      {:.3f}   
'''

    # IDS results
    ids1 = getIDSData(goalState)
    ids2 = getIDSData(goalState2)
    ids3 = getIDSData(goalState3)
    

    # H1 results
    aStar1 = getAStarData(goalState, h1_8p)
    aStar2 = getAStarData(goalState2, h1_8p)
    aStar3 = getAStarData(goalState3, h1_8p)
      

    # H2 results
    aStar4 = getAStarData(goalState, h2_8p)
    aStar5 = getAStarData(goalState2, h2_8p)
    aStar6 = getAStarData(goalState3, h2_8p)
    
    
    # H3 results
    aStar7 = getAStarData(goalState, h3_8p)
    aStar8 = getAStarData(goalState2, h3_8p)
    aStar9 = getAStarData(goalState3, h3_8p)
   
 
    mformat = mformat.format(goalState,
                             goalState2,
                             goalState3,
                             # IDS
                             ids1[0], ids1[1], ids1[2],
                             ids2[0], ids2[1], ids2[2],
                             ids3[0], ids3[1], ids3[2],
                             # H1
                             aStar1[0], aStar1[1], aStar1[2],
                             aStar2[0], aStar2[1], aStar2[2],
                             aStar3[0], aStar3[1], aStar3[2],
                             # H2
                             aStar4[0], aStar4[1], aStar4[2],
                             aStar5[0], aStar5[1], aStar5[2],
                             aStar6[0], aStar6[1], aStar6[2],
                             # H3
                             aStar7[0], aStar7[1], aStar7[2],
                             aStar8[0], aStar8[1], aStar8[2],
                             aStar9[0], aStar9[1], aStar9[2])

    print(mformat)
    return


In [27]:
runExperiment(goalState, goalState2, goalState3,h=[h1_8p, h2_8p, h3_8p])


            [1, 2, 3, 4, 0, 5, 6, 7, 8]    [1, 2, 3, 4, 5, 8, 6, 0, 7]    [1, 0, 3, 4, 5, 8, 2, 6, 7] 
Algorithm    Depth  Nodes       EBF              Depth  Nodes       EBF              Depth  Nodes       EBF          
     IDS       0      0       0.000              3      43      3.086              11      225850      2.954         
    A*h1       0      38       0.000              3      116      4.488              11      643246      3.263
    A*h2       0      0       0.000              3      51      3.297              11      100046      2.733
    A*h3       0      0       0.000              3      9      1.578              11      5232      2.049   



In [28]:
%run -i A3grader.py


Testing actionsF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your actionsF_8p correctly returned [('left', 1), ('right', 1), ('up', 1)]

Testing takeActionF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], (up, 1))

--- 5/5 points. Your takeActionsF_8p correctly returned ([1, 2, 3, 4, 0, 6, 7, 5, 8], 1)

Testing goalTestF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your goalTestF_8p correctly True

Testing aStarSearch([1, 2, 3, 4, 5, 6, 7, 0, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]),
                     lambda s: h1_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]))

--- 20/20 points. Your search correctly returned ([[1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 0, 6, 7, 5, 8], [1, 2, 3, 0, 4, 6, 7, 5, 8], [0, 2, 3, 1, 4, 6, 7, 5, 8]], 3)

Testing iterativeDeepeningSearch([5, 2, 8, 0, 1, 4, 3, 7, 6], 
                                 [0, 2, 3, 1, 4,  6, 7, 5, 8],
                            